## BaseDeviceCollector - Network Device Interaction

The `base.py` script provides a clean, abstract base class for network device management using Netmiko.

### Key Capabilities:

**Connection Management:**
- `connect()` - Establishes SSH connection to network devices
- `disconnect()` - Safely closes connections (exits config mode if needed)
- `is_connected()` - Checks connection status
- Context manager support for automatic connect/disconnect

**Command Execution:**
- `send_show_command()` - Execute read-only show commands
- `send_config_set()` - Apply configuration changes (auto enters/exits config mode)

**Safety Features:**
- `save_config()` - Saves running config to startup config
- Prompt detection and caching
- Automatic privilege escalation (enable mode)

**Design Pattern:**
- Abstract base class (ABC) requiring subclasses to define `device_type`
- Flexible credential management with timeout/delay controls
- Clean error handling and status reporting

## Collector - Cisco IOS Network Observability

The `collector.py` extends `BaseDeviceCollector` to provide comprehensive Layer 1-7 observability for Cisco IOS devices.

### Inheritance & Design:
- **Inherits** all connection management from `BaseDeviceCollector` (connect, disconnect, context manager)
- **Adds** device-specific observability methods on top
- **Auto-detects** device type (telnet vs SSH) based on port number

### Layer 2 Observability:
- `get_vlan_brief()` - VLAN information with port assignments
- `get_mac_address_table()` - MAC address learning table
- `get_trunk_interfaces()` - Trunk configuration and VLAN status
- `get_spanning_tree_summary()` - STP configuration and per-VLAN statistics

### Layer 3 Observability:
- `get_interface_brief()` - IP addresses and interface status
- `get_cdp_neighbors()` - CDP neighbor discovery with topology info
- `get_ospf_neighbors()` - OSPF adjacency information

### Device Health:
- `get_device_info()` - Hardware/software version
- `get_cpu_usage()` - CPU utilization statistics
- `get_memory_usage()` - Memory utilization statistics

### Clean vs Raw Output:
- `clean=True` (default): Parsed structured data (lists/dicts)
- `clean=False`: Raw device output (useful for debugging)

**Usage Pattern:** Use context manager (`with Collector(...) as collector:`) for automatic connection lifecycle management!

In [10]:
# Collector Test - Auto-Connect/Disconnect with Context Manager
from tools.collector import Collector

# Context manager handles connect/disconnect automatically
with Collector(
    device_id="MANAGEMENT",
    host="192.168.56.101",
    port=5010,
    credentials={"username": "matin", "password": "cisco", "secret": "cisco"}
) as collector:
    # Auto-connected on entry
    
    # Layer 3: Interface information
    print("=== Interface Brief ===")
    interfaces = collector.get_interface_brief()
    for intf in interfaces[:2]:
        print(f"{intf['interface']}: {intf['ip_address']} - {intf['status']}/{intf['protocol']}")
    
    # Layer 3: CDP neighbors
    print("\n=== CDP Neighbors ===")
    neighbors = collector.get_cdp_neighbors()
    for neighbor in neighbors[:2]:
        print(f"{neighbor['neighbor_device']} via {neighbor['local_interface']}")
    
    # Auto-disconnected on exit

=== Interface Brief ===
GigabitEthernet0/0: unassigned - down/down
GigabitEthernet0/1: unassigned - up/up

=== CDP Neighbors ===


In [6]:
interfaces

[{'interface': 'GigabitEthernet0/0',
  'ip_address': 'unassigned',
  'ok': 'YES',
  'method': 'unset',
  'status': 'down',
  'protocol': 'down'},
 {'interface': 'GigabitEthernet0/1',
  'ip_address': 'unassigned',
  'ok': 'YES',
  'method': 'unset',
  'status': 'up',
  'protocol': 'up'},
 {'interface': 'GigabitEthernet0/2',
  'ip_address': 'unassigned',
  'ok': 'YES',
  'method': 'unset',
  'status': 'up',
  'protocol': 'up'},
 {'interface': 'GigabitEthernet0/3',
  'ip_address': 'unassigned',
  'ok': 'YES',
  'method': 'unset',
  'status': 'up',
  'protocol': 'up'},
 {'interface': 'GigabitEthernet1/0',
  'ip_address': 'unassigned',
  'ok': 'YES',
  'method': 'unset',
  'status': 'down',
  'protocol': 'down'},
 {'interface': 'GigabitEthernet1/1',
  'ip_address': 'unassigned',
  'ok': 'YES',
  'method': 'unset',
  'status': 'down',
  'protocol': 'down'},
 {'interface': 'GigabitEthernet1/2',
  'ip_address': 'unassigned',
  'ok': 'YES',
  'method': 'unset',
  'status': 'down',
  'protocol':

In [7]:
# Context manager handles connect/disconnect automatically
with Collector(
    device_id="MANAGEMENT",
    host="192.168.56.101",
    port=5010,
    credentials={"username": "matin", "password": "cisco", "secret": "cisco"}
) as collector:
    # Auto-connected on entry
    
    # Layer 3: Interface information
    print("=== Interface Brief ===")
    interfaces = collector.get_interface_brief(clean=False)
    print(interfaces)

    # Layer 3: CDP neighbors
    print("\n=== CDP Neighbors ===")
    neighbors = collector.get_cdp_neighbors(clean=False)
    print(neighbors)

    # Auto-disconnected on exit

=== Interface Brief ===
{'lines': ['Interface              IP-Address      OK? Method Status                Protocol', 'GigabitEthernet0/0     unassigned      YES unset  down                  down    ', 'GigabitEthernet0/1     unassigned      YES unset  up                    up      ', 'GigabitEthernet0/2     unassigned      YES unset  up                    up      ', 'GigabitEthernet0/3     unassigned      YES unset  up                    up      ', '', 'GigabitEthernet1/0     unassigned      YES unset  down                  down    ', 'GigabitEthernet1/1     unassigned      YES unset  down                  down    ', 'GigabitEthernet1/2     unassigned      YES unset  down                  down    ', 'GigabitEthernet1/3     unassigned      YES unset  down                  down    ', 'GigabitEthernet2/0     unassigned      YES unset  down                  down    ', 'GigabitEthernet2/1     unassigned      YES unset  down                  down    ', 'GigabitEthernet2/2     unassigned   

In [8]:
interfaces

{'lines': ['Interface              IP-Address      OK? Method Status                Protocol',
  'GigabitEthernet0/0     unassigned      YES unset  down                  down    ',
  'GigabitEthernet0/1     unassigned      YES unset  up                    up      ',
  'GigabitEthernet0/2     unassigned      YES unset  up                    up      ',
  'GigabitEthernet0/3     unassigned      YES unset  up                    up      ',
  '',
  'GigabitEthernet1/0     unassigned      YES unset  down                  down    ',
  'GigabitEthernet1/1     unassigned      YES unset  down                  down    ',
  'GigabitEthernet1/2     unassigned      YES unset  down                  down    ',
  'GigabitEthernet1/3     unassigned      YES unset  down                  down    ',
  'GigabitEthernet2/0     unassigned      YES unset  down                  down    ',
  'GigabitEthernet2/1     unassigned      YES unset  down                  down    ',
  'GigabitEthernet2/2     unassigned   

## Graph Database Pipeline - Complete Data Flow

### 1. graph/base.py - Graph Utilities & Device Management

**Purpose:** Centralized utilities for Neo4j operations and device configuration loading.

**Key Functions:**

**Device Management:**
- `load_devices()` - Reads `graph/config/devices.yaml` and returns device inventory
- Returns dict: `{hostname: {type, mgmt_ip, mgmt_port, credentials, enabled}}`
- Used by `network_fetch.py` (data collection) and `feed_snapshot.py` (ingestion)

**Neo4j Connection Management:**
- `load_neo4j_connection()` - Reads `graph/config/neo4j.yaml` for DB credentials
- `create_driver()` - Creates Neo4j driver with authentication
- `GraphClient` - Context manager wrapper for Neo4j driver lifecycle
  - Auto-connects on entry (`__enter__`)
  - Auto-closes on exit (`__exit__`)

**Database Operations:**
- `clear_db()` - Delete all nodes and relationships (clean slate)
- `create_indexes()` - Create performance indexes on Device, Interface, VLAN, Snapshot nodes
- `create_devices()` - Create Device nodes from YAML (baseline skeleton)
- `build_baseline()` - Initialize graph with device skeletons + indexes

**Utility Functions:**
- `get_device_count()` - Count Device nodes in graph
- `list_snapshots()` - List available JSON snapshot files in `graph/snapshots/`

---

### 2. graph/network_fetch.py - Network Data Collector

**Purpose:** Connect to live network devices and collect real-time topology/configuration data.

**NetworkFetcher Class:**

**Initialization:**
- Loads device inventory from `devices.yaml` via `load_devices()`
- Creates list of all enabled network devices

**Data Collection Methods:**

1. **`fetch_device(hostname, device_config)`** - Collect data from single device:
   - Creates `Collector` instance with credentials
   - Uses context manager for auto-connect/disconnect
   - **For Routers:** Fetches interfaces, CDP neighbors, OSPF neighbors
   - **For Switches:** Fetches interfaces, CDP, VLANs, trunks, MAC table, STP, OSPF (if L3)
   - Returns structured dict with all collected data

2. **`fetch_all()`** - Collect data from entire network:
   - Generates single `snapshot_id` (ISO timestamp) for consistency
   - Iterates through all enabled devices
   - Calls `fetch_device()` for each device
   - Aggregates all device data into single JSON structure
   - Writes to `graph/snapshots/network_YYYY-MM-DDTHH-MM-SS.json`
   - Returns complete network snapshot dict

3. **`run()`** - Entry point that executes `fetch_all()`

**Output Format:**
```json
{
  "snapshot_id": "2024-01-15T10:30:00.123456",
  "devices": [
    {
      "hostname": "CORE-SW1",
      "type": "switch",
      "ip_address": "10.10.10.2",
      "interfaces": [...],
      "cdp_neighbors": [...],
      "vlans": [...],
      "trunks": [...],
      "mac_addresses": [...],
      "spanning_tree": {...},
      "ospf_neighbors": [...]
    },
    ...
  ]
}
```

**Key Features:**
- Single snapshot ID ensures temporal consistency
- Type-aware data collection (router vs switch)
- Graceful error handling (continues if device fails)
- Uses Collector with context manager (auto-connection management)

---

### 3. graph/feed_snapshot.py - Neo4j Ingestion Engine

**Purpose:** Ingest JSON snapshots into Neo4j graph database with full topology modeling.

**Three-Phase Ingestion Process:**

**Phase 0: Create Snapshot Node**
```cypher
CREATE (s:Snapshot {id, timestamp, device_count})
```
- Tracks when data was collected
- Links all nodes to snapshot for temporal queries
- Enables time-series analysis of network changes

**Phase 1: Create Nodes**

1. **Interface Nodes:**
   - Creates `Interface` node for each device interface
   - ID format: `hostname:interface_name` (e.g., "CORE-SW1:Gi0/1")
   - Properties: name, ip_address, status, protocol, method, ok
   - Links to Device via `HAS_INTERFACE` relationship
   - Tagged with `snapshot_id`

2. **Switch Data Storage:**
   - Stores Layer 2 data as JSON properties on Device nodes
   - `vlans` - VLAN configurations (serialized JSON array)
   - `mac_addresses` - MAC address table (serialized JSON array)
   - `spanning_tree` - STP config and stats (serialized JSON object)
   - `trunks` - Trunk interface configs (serialized JSON array)
   - Tagged with `snapshot_id`

**Phase 2: Create Relationships**

1. **CDP Physical Connections** (`CONNECTED_TO`):
   - Maps CDP neighbor data to Interface-to-Interface connections
   - Builds lookup: `iface_by_name[(hostname, interface)] → interface_data`
   - Validates both local and remote interfaces exist
   - Creates bidirectional physical links
   - Properties: protocol='CDP', neighbor_ip, local/remote status/protocol
   - Tagged with `snapshot_id`

2. **OSPF Logical Connections** (`OSPF_NEIGHBOR`):
   - Maps OSPF neighbor IPs to Device hostnames
   - Builds lookup: `device_by_ip[ip_address] → hostname`
   - Creates Device-to-Device routing relationships
   - Properties: neighbor_id, state (FULL/DR), priority, dead_time, interface
   - Tagged with `snapshot_id`

**Data Validation:**
- Only creates CDP links if both interfaces exist in graph
- Only creates OSPF links if neighbor IP can be mapped to device
- Handles partial data gracefully (missing fields)

**Output Summary:**
Returns dict with ingestion statistics:
```python
{
  "snapshot_id": "2024-01-15T10:30:00.123456",
  "devices": 6,
  "interfaces": 48,
  "cdp_connections": 12,
  "ospf_connections": 4
}
```

**Usage:**
```bash
python feed_snapshot.py graph/snapshots/network_2024-01-15T10-30-00.json
```

---

### Complete End-to-End Workflow:

```
1. devices.yaml             → Device inventory (static config)
         ↓
2. network_fetch.py         → Connects to live devices via Collector
         ↓                     (uses BaseDeviceCollector context manager)
3. JSON Snapshot            → Timestamped file: graph/snapshots/network_*.json
         ↓                     Single snapshot_id for entire network
4. feed_snapshot.py         → Parses JSON and ingests to Neo4j
         ↓                     Phase 0: Snapshot node
         ↓                     Phase 1: Interface nodes + switch data
         ↓                     Phase 2: CDP + OSPF relationships
5. Neo4j Graph Database     → Queryable network topology with Cypher
```

**Graph Schema:**
```
(Snapshot)                           [Temporal tracking node]
    ↓
(Device)-[:HAS_INTERFACE]->(Interface)       [Device to interfaces]
    ↓
(Interface)-[:CONNECTED_TO]->(Interface)     [CDP physical links]
    ↓
(Device)-[:OSPF_NEIGHBOR]->(Device)          [OSPF routing adjacencies]
```

**All nodes/relationships tagged with `snapshot_id` for temporal queries!**

**Example Cypher Queries:**
```cypher
// Find all devices
MATCH (d:Device) RETURN d.hostname, d.type

// Find physical topology via CDP
MATCH (i1:Interface)-[r:CONNECTED_TO]->(i2:Interface)
RETURN i1.id, i2.id, r.protocol

// Find OSPF neighbors
MATCH (d1:Device)-[r:OSPF_NEIGHBOR]->(d2:Device)
RETURN d1.hostname, d2.hostname, r.state

// Get specific snapshot
MATCH (s:Snapshot {id: "2024-01-15T10:30:00"})
MATCH (d:Device {snapshot_id: s.id})
RETURN d
```

## Graph Query Toolkit - graph/cypher.py

### Purpose: 
LangChain-integrated Cypher query helpers for intelligent network topology analysis and troubleshooting.

---

### Architecture Design

**Three-Layer Architecture:**

1. **Internal Helpers** (Private utilities)
2. **Query Helpers** (Pure Python functions)
3. **Tool Wrappers** (LangChain `@tool` decorators for agent integration)

---

### 1. Internal Helpers

**`_deduplicate_records(records)`**
- Removes duplicate query results
- Uses JSON serialization for consistent hashing
- Ensures clean, unique result sets

**`_run_query(cypher, params, timeout, deduplicate)`**
- Centralized query execution function
- Uses `GraphClient` context manager for auto-connection management
- Configurable timeout (default: 30s)
- Optional deduplication (default: enabled)
- Returns list of dicts with query results

---

### 2. Query Helper Functions (Pure Functions)

**Device Management:**
- `list_devices()` - List all devices with hostname, type, IP
- `count_interfaces()` - Count interfaces per device (sorted by count)

**Topology Discovery:**
- `show_topology()` - Full CDP physical topology (all connections)
- `show_cdp_neighbors_device(device)` - CDP neighbors for specific device
- `show_interfaces_connected_device(device)` - All connected interfaces for device

**Interface Status:**
- `show_up_interfaces()` - All up/up interfaces (global)
- `show_up_interfaces_device(device)` - Up/up interfaces on specific device
- `find_down_interfaces()` - All down or problematic interfaces (global)

**OSPF Routing:**
- `show_ospf_neighbors()` - All OSPF adjacencies (global)
- `show_ospf_neighbors_device(device)` - OSPF neighbors for specific device

**Path Analysis:**
- `show_shortest_path(device1, device2)` - One shortest path between two devices
- `show_all_paths(device1, device2)` - All shortest paths between two devices

---

### 3. LangChain Tool Wrappers

**Purpose:** Make Cypher queries available to AI agents via `@tool` decorator.

**Active Tools (Enabled for agents):**
- `list_devices_tool()` - Device inventory
- `show_ospf_neighbors_tool()` - OSPF adjacencies
- `show_interfaces_connected_device_tool(device)` - Device connections
- `show_cdp_neighbors_device_tool(device)` - CDP neighbors
- `show_ospf_neighbors_device_tool(device)` - OSPF neighbors per device
- `show_shortest_path_tool(device1, device2)` - Single shortest path
- `show_all_paths_tool(device1, device2)` - All shortest paths

**Commented Tools:** Some tools are commented out to limit agent tool selection complexity. Can be enabled as needed.

---

### Cypher Query Patterns

**1. Device Listing:**
```cypher
MATCH (d:Device)
RETURN d.hostname, d.type, d.ip_address
ORDER BY d.hostname
```

**2. Topology Discovery (CDP Physical):**
```cypher
MATCH (d1:Device)-[:HAS_INTERFACE]->(i1:Interface)
      -[r:CONNECTED_TO]->(i2:Interface)
      <-[:HAS_INTERFACE]-(d2:Device)
RETURN d1.hostname, i1.name, d2.hostname, i2.name, r.protocol
```

**3. OSPF Adjacencies (Logical):**
```cypher
MATCH (d:Device)-[r:OSPF_NEIGHBOR]->(n:Device)
RETURN d.hostname, n.hostname, r.state, r.neighbor_id
```

**4. Interface Status Filtering:**
```cypher
MATCH (d:Device)-[:HAS_INTERFACE]->(i:Interface)
WHERE i.status = 'up' AND i.protocol = 'up'
RETURN d.hostname, i.name, i.ip_address
```

**5. Shortest Path Analysis:**
```cypher
MATCH p = shortestPath(
  (a:Device {hostname: $device1})
  -[:HAS_INTERFACE|CONNECTED_TO*]-
  (b:Device {hostname: $device2})
)
RETURN nodes(p)
```

---

### Tool Integration with AI Agents

**How Agents Use These Tools:**

1. **Agent receives user query** (e.g., "What devices are connected to CORE-SW1?")
2. **Agent selects appropriate tool** (`show_interfaces_connected_device_tool`)
3. **Tool executes Cypher query** via `_run_query()` helper
4. **Results returned to agent** as structured list of dicts
5. **Agent formats response** for user in natural language

**Example Agent Flow:**
```
User: "Show me the path from EDGE-R1 to ACC-SW2"
  ↓
Agent: Calls show_shortest_path_tool("EDGE-R1", "ACC-SW2")
  ↓
Cypher Query: MATCH p = shortestPath(...)
  ↓
Neo4j: Returns path nodes
  ↓
Agent: "The path is: EDGE-R1 → Gi0/1 → CORE-SW1 → Gi0/3 → ACC-SW2"
```

---

### Key Design Principles

**1. Separation of Concerns:**
- Pure functions (query helpers) separate from tool wrappers
- Can use functions directly in Python or via agent tools

**2. Consistent Error Handling:**
- Uses GraphClient context manager (auto-cleanup)
- Configurable timeouts prevent hanging queries
- Deduplication ensures clean results

**3. Agent-Friendly:**
- Clear tool names with descriptive docstrings
- Predictable input/output (strings → list of dicts)
- Selective tool exposure (commented tools reduce complexity)

**4. Extensibility:**
- Easy to add new query helpers
- Simple pattern: write function → add `@tool` wrapper
- Automatic export via `__all__`

---

### Use Cases

**Network Operations:**
- "Which devices are down?"
- "Show me the topology"
- "What's connected to CORE-SW1?"

**Troubleshooting:**
- "Find all down interfaces"
- "Show OSPF neighbors for EDGE-R1"
- "What's the path between Device A and B?"

**Planning & Analysis:**
- "How many interfaces does each device have?"
- "Show all paths between core switches"
- "List all OSPF adjacencies"

In [11]:
# Cypher Tools Demo - Direct Functions & LangChain Tools
import sys
from pathlib import Path

# Add graph module to path
graph_path = Path.cwd() / "graph"
if str(graph_path) not in sys.path:
    sys.path.insert(0, str(graph_path))

from graph.cypher import (
    list_devices,
    show_topology,
    show_cdp_neighbors_device,
    show_ospf_neighbors,
    show_shortest_path,
    list_devices_tool,
    show_interfaces_connected_device_tool
)

print("=" * 70)
print("1. DIRECT FUNCTION USAGE - List All Devices")
print("=" * 70)
devices = list_devices()
for device in devices:
    print(f"  {device['host']:15} | {device['type']:8} | {device['ip']}")

print("\n" + "=" * 70)
print("2. DIRECT FUNCTION USAGE - Show CDP Topology")
print("=" * 70)
topology = show_topology()
for link in topology[:5]:  # Show first 5 links
    print(f"  {link['from']:12} [{link['from_if']:12}] --{link['protocol']}--> [{link['to_if']:12}] {link['to']}")

print("\n" + "=" * 70)
print("3. DIRECT FUNCTION USAGE - CDP Neighbors for CORE-SW1")
print("=" * 70)
cdp_neighbors = show_cdp_neighbors_device("CORE-SW1")
for neighbor in cdp_neighbors:
    print(f"  {neighbor['local_iface']:12} --> {neighbor['neighbor_device']:12} ({neighbor['neighbor_iface']})")

print("\n" + "=" * 70)
print("4. DIRECT FUNCTION USAGE - OSPF Neighbors (All)")
print("=" * 70)
ospf = show_ospf_neighbors()
for nbr in ospf:
    print(f"  {nbr['local']:12} --> {nbr['neighbor']:12} | State: {nbr['state']:10} | Interface: {nbr['local_if']}")

print("\n" + "=" * 70)
print("5. DIRECT FUNCTION USAGE - Shortest Path")
print("=" * 70)
path = show_shortest_path("EDGE-R1", "ACC-SW2")
if path:
    print(f"  Path: {' -> '.join(path[0]['path_nodes'])}")

print("\n" + "=" * 70)
print("6. LANGCHAIN TOOL USAGE - List Devices Tool")
print("=" * 70)
# Using .invoke() method for LangChain tools
tool_result = list_devices_tool.invoke({})
print(f"  Tool returned {len(tool_result)} devices")
for device in tool_result[:3]:
    print(f"  {device}")

print("\n" + "=" * 70)
print("7. LANGCHAIN TOOL USAGE - Connected Interfaces Tool")
print("=" * 70)
# Tool with parameter
tool_result = show_interfaces_connected_device_tool.invoke({"device": "CORE-SW1"})
print(f"  Tool returned {len(tool_result)} connections")
for conn in tool_result[:3]:
    print(f"  {conn}")

print("\n" + "=" * 70)
print("Demo Complete!")
print("=" * 70)

c:\Matin\codes\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


1. DIRECT FUNCTION USAGE - List All Devices
  ACC-SW1         | switch   | 10.10.10.4
  ACC-SW2         | switch   | 10.10.10.5
  CORE-SW1        | switch   | 10.10.10.2
  CORE-SW2        | switch   | 10.10.10.3
  EDGE-R1         | router   | 10.10.10.1
  MANAGEMENT      | switch   | 10.10.10.10

2. DIRECT FUNCTION USAGE - Show CDP Topology
  ACC-SW1      [GigabitEthernet0/1] --CDP--> [GigabitEthernet1/0] CORE-SW1
  ACC-SW1      [GigabitEthernet0/2] --CDP--> [GigabitEthernet1/0] CORE-SW2
  ACC-SW2      [GigabitEthernet0/1] --CDP--> [GigabitEthernet1/1] CORE-SW1
  ACC-SW2      [GigabitEthernet0/2] --CDP--> [GigabitEthernet1/1] CORE-SW2
  CORE-SW1     [GigabitEthernet1/0] --CDP--> [GigabitEthernet0/1] ACC-SW1

3. DIRECT FUNCTION USAGE - CDP Neighbors for CORE-SW1
  GigabitEthernet1/0 --> ACC-SW1      (GigabitEthernet0/1)
  GigabitEthernet1/1 --> ACC-SW2      (GigabitEthernet0/1)
  GigabitEthernet0/2 --> CORE-SW2     (GigabitEthernet0/2)
  GigabitEthernet0/3 --> CORE-SW2     (GigabitEther

INFO:neo4j.notifications:Received notification from DBMS server: <GqlStatusObject gql_status='03N91', status_description="info: unbounded variable length pattern. The provided pattern '(a:Device {hostname: $device1})-[:HAS_INTERFACE|CONNECTED_TO*]-(b:Device {hostname: $device2})' is unbounded. Shortest path with an unbounded pattern may result in long execution times. Use an upper limit (e.g. '[*..5]') on the number of node hops in your pattern.", position=<SummaryInputPosition line=2, column=32, offset=32>, raw_classification='PERFORMANCE', classification=<NotificationClassification.PERFORMANCE: 'PERFORMANCE'>, raw_severity='INFORMATION', severity=<NotificationSeverity.INFORMATION: 'INFORMATION'>, diagnostic_record={'_classification': 'PERFORMANCE', '_severity': 'INFORMATION', '_position': {'offset': 32, 'line': 2, 'column': 32}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: '\n        MATCH p = shortestPath((a:Device {hostname: $device1})-[:HAS_INTERFACE|

  Path: EDGE-R1 (10.10.10.1) -> IF:GigabitEthernet0/0 -> IF:GigabitEthernet0/1 -> MANAGEMENT (10.10.10.10) -> IF:GigabitEthernet0/2 -> IF:GigabitEthernet0/1 -> CORE-SW1 (10.10.10.2) -> IF:GigabitEthernet1/1 -> IF:GigabitEthernet0/1 -> ACC-SW2 (10.10.10.5)

6. LANGCHAIN TOOL USAGE - List Devices Tool
  Tool returned 6 devices
  {'host': 'ACC-SW1', 'type': 'switch', 'ip': '10.10.10.4'}
  {'host': 'ACC-SW2', 'type': 'switch', 'ip': '10.10.10.5'}
  {'host': 'CORE-SW1', 'type': 'switch', 'ip': '10.10.10.2'}

7. LANGCHAIN TOOL USAGE - Connected Interfaces Tool
  Tool returned 5 connections
  {'local_iface': 'GigabitEthernet1/0', 'remote_device': 'ACC-SW1', 'remote_iface': 'GigabitEthernet0/1', 'protocol': 'CDP'}
  {'local_iface': 'GigabitEthernet1/1', 'remote_device': 'ACC-SW2', 'remote_iface': 'GigabitEthernet0/1', 'protocol': 'CDP'}
  {'local_iface': 'GigabitEthernet0/2', 'remote_device': 'CORE-SW2', 'remote_iface': 'GigabitEthernet0/2', 'protocol': 'CDP'}

Demo Complete!


# Future Development: GNN Implementation for Network Topology Analysis

## Executive Summary
Based on the current Neo4j graph structure and network snapshots, Graph Neural Networks (GNNs) can be integrated to create embeddings, detect anomalies, and predict network issues.

---

## 1. What is a GNN and How Does it Differ from Transformers?

### GNN Architecture:
- GNNs are NOT Transformers, but share conceptual similarities
- **GNNs use message passing** - each node aggregates information from direct neighbors layer by layer
- **Transformers use self-attention** - every token attends to every other token globally
- Key insight: Transformers = GNNs on fully connected graphs; GNNs work on sparse graphs (like network topology)

### Message Passing in GNNs:
```
h_v^(k+1) = UPDATE(h_v^(k), AGGREGATE({h_u^(k) : u ∈ N(v)}))

Where:
- h_v^(k) = node embedding at layer k
- N(v) = neighbors of node v
- Information flows through edges, respecting actual network topology
```

### Why GNN for Network Topology?
- Network has sparse connections (routers/switches don't connect to everything)
- GNNs excel at capturing local relationships and hierarchical structures
- GNNs avoid computational cost of full self-attention - perfect for network graphs

---

## 2. How to Convert Snapshots to Embeddings

### Current Snapshot Structure:
```json
{
  "snapshot_id": "2026-01-11T10:30:00",
  "devices": [
    {
      "hostname": "EDGE-R1",
      "type": "router",
      "interfaces": [...],
      "cdp_neighbors": [...],
      "ospf_neighbors": [...]
    }
  ]
}
```

### Embedding Pipeline:

#### Step 1: Extract Graph from Neo4j
```cypher
MATCH (d1:Device)-[r:CONNECTED_TO]->(d2:Device)
RETURN d1.hostname, d2.hostname,
       r.local_interface, r.remote_interface
```

#### Step 2: Create Node Features
```python
# Example node features for EDGE-R1
node_features = {
    "device_type": [1, 0],  # [router, switch] one-hot
    "num_interfaces": 8,
    "num_vlans": 0,
    "num_ospf_neighbors": 2,
    "num_cdp_neighbors": 3,
    "interface_status": [1, 1, 0, 1, ...],  # up/down per interface
    "vlan_counts": [...]
}
```

#### Step 3: Build PyTorch Geometric Graph
```python
import torch
from torch_geometric.data import Data

# Nodes: devices
x = torch.tensor([device_features], dtype=torch.float)

# Edges: CONNECTED_TO relationships
edge_index = torch.tensor([
    [0, 1, 2],  # source nodes
    [1, 2, 0]   # target nodes
], dtype=torch.long)

# Edge features: interface properties
edge_attr = torch.tensor([[speed, bandwidth, status], ...])

graph_data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
```

---

## 3. Best GNN Implementation Options

### Option 1: Neo4j GDS + HashGNN (Recommended for Production)

#### Why HashGNN?
- 2-4 orders of magnitude faster than learning-based GNNs
- No training required - generates embeddings directly from graph structure
- Built into Neo4j Graph Data Science library
- Perfect for Neo4j backend

#### Implementation:
```python
from graphdatascience import GraphDataScience

gds = GraphDataScience("bolt://localhost:7687", auth=("neo4j", "password"))

# Project network graph
G = gds.graph.project("network_topology", "Device", "CONNECTED_TO")

# Generate embeddings with HashGNN
result = gds.hashgnn.mutate(
    G,
    mutateProperty="embedding",
    embeddingDimension=128,
    iterations=3
)

# Read embeddings
embeddings = gds.graph.nodeProperties.stream(G, "embedding")
```

### Option 2: PyTorch Geometric + GraphSAGE (Best for Anomaly Detection)

#### Why GraphSAGE?
- Inductive learning - handles new devices added to network
- Samples and aggregates from neighbors (fits CDP/OSPF topology)
- Used by Airbus for anomaly detection

#### Implementation:
```python
import torch
from torch_geometric.nn import SAGEConv

class NetworkGNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x  # Node embeddings

model = NetworkGNN(in_channels=50, hidden_channels=64, out_channels=32)
```

### Option 3: Autoencoder GNN (NVIDIA Approach)

#### Why Autoencoder?
- Used by NVIDIA for NetFlow anomaly detection
- Learns to reconstruct "normal" network states
- High reconstruction error = anomaly

#### Workflow:
```python
# Train autoencoder on normal snapshots
normal_embeddings = encoder(graph_data)  # 128-dim
reconstructed = decoder(normal_embeddings)

# At inference, compare reconstruction
new_snapshot_embedding = encoder(new_graph)
reconstructed_new = decoder(new_snapshot_embedding)
anomaly_score = mse_loss(new_graph.x, reconstructed_new)

if anomaly_score > 0.5:
    alert("Network configuration anomaly detected!")
```

---

## 4. What You Can Do with Embeddings

### Use Case 1: Anomaly Detection (Compare Embeddings)
```python
# Train on 100 normal snapshots
normal_embeddings = [gnn(snapshot) for snapshot in normal_data]
centroid = mean(normal_embeddings)  # "Normal" baseline

# New snapshot arrives
new_embedding = gnn(new_snapshot)
distance = ||new_embedding - centroid||  # L2 distance

if distance > threshold:
    alert("Anomaly detected!")  # Config drift, link failure, etc.
```
**Use:** Detect when network state deviates from normal (unexpected VLAN changes, interface down)

### Use Case 2: Time-Series Forecasting (Predict Future States)
```python
# Collect embeddings over time
embeddings_history = [
    gnn(snapshot_t0),  # [128 dims]
    gnn(snapshot_t1),
    gnn(snapshot_t2),
    ...
]

# Train LSTM/Transformer on embedding sequences
lstm = LSTM(input_size=128, hidden_size=64)
predicted_embedding = lstm(embeddings_history)

# Decode back to graph properties
predicted_failures = decoder(predicted_embedding)
# Output: "Interface Gi0/1 on EDGE-R1 likely to fail in 2 hours"
```
**Use:** Predict interface failures, bandwidth exhaustion, routing loops

### Use Case 3: Clustering & Similarity (Compare Devices)
```python
# Get embeddings for all devices
device_embeddings = gnn(full_network_graph)  # [num_devices, 128]

# Find similar configurations
similarity_matrix = cosine_similarity(device_embeddings)

# Example output:
# EDGE-R1 ↔ EDGE-R2: 0.95 similarity (nearly identical configs)
# CORE-SW1 ↔ ACC-SW1: 0.42 similarity (very different)
```
**Use:** Find misconfigured devices, recommend template configs, detect shadow IT

---

## 5. Embedding Comparison Workflow

### Temporal Comparison (Snapshot-to-Snapshot)
```python
# Compare network state over time
embedding_t0 = gnn_encode(snapshot_2026_01_11_10_00)  # [num_devices, 128]
embedding_t1 = gnn_encode(snapshot_2026_01_11_11_00)  # [num_devices, 128]

# Cosine similarity per device
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(embedding_t0, embedding_t1)

# Devices with low similarity changed significantly
changed_devices = np.where(similarity.diagonal() < 0.9)[0]
```

### Anomaly Detection via Distance
```python
# Compute centroid (average normal state)
centroid = np.mean(normal_embeddings, axis=0)

# New snapshot arrives
new_emb = gnn_encode(new_snapshot)
distance = np.linalg.norm(new_emb - centroid, axis=1)  # L2 distance per device

# Flag devices far from normal
anomalies = np.where(distance > threshold)[0]
```

---

## 6. Recommended Implementation Roadmap

### Phase 1: Quick Start with Neo4j GDS (1-2 days)
1. Install Neo4j Graph Data Science plugin
2. Use HashGNN to generate embeddings
3. Store embeddings as node properties
4. Compare embeddings across snapshots

### Phase 2: PyTorch Geometric for Anomaly Detection (1 week)
1. Export Neo4j graph to PyG format
2. Extract node features from snapshots
3. Train GraphSAGE on normal network states
4. Deploy inference pipeline for new snapshots

### Phase 3: Advanced - Autoencoder GNN (2 weeks)
1. Build Graph U-Net autoencoder
2. Train on historical snapshots
3. Set up real-time anomaly scoring
4. Integrate alerts into agent

---

## 7. Integration with Current System

### Workflow:
```
network_fetch.py (snapshots)
    ↓
Neo4j (graph topology)
    ↓
GNN Encoder (PyG or Neo4j GDS)
    ↓
Embeddings (128-dim vectors per device)
    ↓
Comparison/Anomaly Detection
    ↓
NetworkAgent (alerts user)
```

### Code Skeleton:
```python
# In agents/network_agent.py
class NetworkAgent:
    def __init__(self):
        self.gnn_encoder = load_gnn_model()
        self.normal_embeddings = load_baseline()

    def detect_anomalies(self, snapshot_path):
        # 1. Load snapshot
        snapshot = json.load(open(snapshot_path))

        # 2. Build PyG graph
        graph = self._snapshot_to_graph(snapshot)

        # 3. Generate embeddings
        embeddings = self.gnn_encoder(graph.x, graph.edge_index)

        # 4. Compare to baseline
        distances = self._compute_distances(embeddings, self.normal_embeddings)

        # 5. Flag anomalies
        anomalies = np.where(distances > self.threshold)[0]

        return {
            "anomalies": [snapshot['devices'][i]['hostname'] for i in anomalies],
            "distances": distances.tolist()
        }
```

---

## Summary: Embeddings = Compressed Network State

### Typical Workflow:
```
Snapshot 1 → GNN → [0.2, 0.8, ..., 0.5]  ─┐
Snapshot 2 → GNN → [0.3, 0.7, ..., 0.6]  ─┤ Compare
Snapshot 3 → GNN → [0.9, 0.1, ..., 0.2]  ─┘ (anomaly!)
                        ↓
            Train LSTM for forecasting
                        ↓
            Predict future anomalies
```

### Bottom Line:
- **Compare embeddings** → Anomaly detection
- **Sequence embeddings** → Forecasting
- **Cluster embeddings** → Similarity analysis

### Recommendation:
Start with Neo4j GDS + HashGNN for quick prototyping (no training needed), then move to PyTorch Geometric + GraphSAGE for production anomaly detection. Current snapshots already contain perfect features (interface counts, VLAN data, neighbor info) for rich node embeddings!

---

## References & Sources

### Graph Neural Networks
- [A Gentle Introduction to Graph Neural Networks](https://distill.pub/2021/gnn-intro/)
- [Graph Neural Network Applications](https://arxiv.org/abs/2005.03675)
- [Transformers are Graph Neural Networks](https://thegradient.pub/transformers-are-graph-neural-networks/)
- [Can Transformer be viewed as a GNN?](https://www.semanticscholar.org/)

### GNN Implementations
- [PyTorch Geometric Documentation](https://pytorch-geometric.readthedocs.io/)
- [PyTorch Geometric GitHub](https://github.com/pyg-team/pytorch_geometric)
- [Neo4j Graph Data Science](https://neo4j.com/docs/graph-data-science/current/)
- [HashGNN Deep-dive - Towards Data Science](https://towardsdatascience.com/)

### Network Anomaly Detection
- [Autoencoder-Based GNNs for Network Anomaly Detection - NVIDIA](https://developer.nvidia.com/)
- [GNNs for Anomaly Detection with Python](https://towardsdatascience.com/)
- [FN-GNN: Graph Embedding for Network Intrusion Detection](https://arxiv.org/abs/)
- [MGF-GNN: Multi-Granularity Graph Fusion for Network Intrusion Detection](https://arxiv.org/abs/)

### Applications
- [PowerGNN: Topology-Aware GNN for Electricity Grids](https://arxiv.org/abs/)
- [GNNs for Next-Generation IoT](https://ieeexplore.ieee.org/)
- [Graph representation learning via enhanced GNNs and transformers](https://arxiv.org/)
